In [1]:
def count_para(model):
    trainable_parameters = list(filter(lambda p: p.requires_grad, model.parameters()))
    n_trainable_parameters = sum([p.numel() for p in trainable_parameters])
    return n_trainable_parameters


In [2]:
import os
os.chdir('/home/students/wittmann/jw-dir/repos/conditional_INNs/colorization_cINN')


In [3]:
import FrEIA
from model import *

/home/students/wittmann/jw-dir/repos/conditional_INNs/colorization_cINN/model.py:174: UserWarning: No loading pretrained weights for conditioning network (./features_pretrained.pt)
  warnings.warn("No loading pretrained weights for conditioning network (./features_pretrained.pt)")
/work/local/venv9/lib/python3.12/site-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


In [4]:
print(count_para(combined_model.inn), 'inn parameters')
print(count_para(combined_model.fc_cond_network), 'cond parameters')
print(count_para(combined_model.feature_network), 'feature parameters')
print(count_para(combined_model), 'total parameters')


print(list(combined_model.inn.modules()).__len__(), 'modules')
print(list(x for x in combined_model.inn.modules() if isinstance(x, (FrEIA.modules.coupling_layers._BaseCouplingBlock, subnet_coupling_layer))).__len__(), 'coupling layers')
print(list(count_para(x) for x in list(x for x in combined_model.inn.modules() if isinstance(x, (FrEIA.modules.coupling_layers._BaseCouplingBlock, subnet_coupling_layer)))))


print((combined_model.inn.dims_in), 'input')
print((combined_model.inn.dims_c), 'cond')


55360784 inn parameters
2361472 cond parameters
32236013 feature parameters
89958269 total parameters
643 modules
38 coupling layers
[112548, 112548, 112548, 112548, 2928, 2928, 530512, 530512, 530512, 530512, 530512, 530512, 46272, 46272, 1236128, 1236128, 1236128, 1236128, 1236128, 1236128, 184704, 184704, 3909952, 3909952, 3909952, 3909952, 3909952, 3909952, 738048, 738048, 2363392, 2363392, 2363392, 2363392, 2363392, 2363392, 2363392, 2363392]
[(2, 64, 64)] input
[(256, 64, 64), (512,)] cond


In [5]:
list(x.shape for x in combined_model.inn(torch.rand(7, 2, 64, 64), [torch.rand(7, 256, 64, 64), torch.rand(7, 512)])[0])

[torch.Size([7, 4096]),
 torch.Size([7, 2048]),
 torch.Size([7, 1536]),
 torch.Size([7, 512])]

In [6]:
list(x.shape for x in combined_model(torch.rand(7,3,256,256)))

[torch.Size([7]), torch.Size([7])]

In [8]:
for x in combined_model.inn.node_list_fwd:
    print(x, x.conditions, x.condition_dims)

InputNode 'inp': [] ->  -> [(2, 64, 64)] [] []
ConditionNode 'fe7a40': [] ->  -> [(256, 64, 64)] [] []
Node 'conv_0': [(2, 64, 64)] -> subnet_coupling_layer -> [(2, 64, 64)] [(ConditionNode 'fe7a40', 0)] [(256, 64, 64)]
Node 'e2b470': [(2, 64, 64)] -> Fixed1x1Conv -> [(2, 64, 64)] [] []
Node 'conv_1': [(2, 64, 64)] -> subnet_coupling_layer -> [(2, 64, 64)] [(ConditionNode 'fe7a40', 0)] [(256, 64, 64)]
Node 'e2b830': [(2, 64, 64)] -> Fixed1x1Conv -> [(2, 64, 64)] [] []
Node 'conv_2': [(2, 64, 64)] -> subnet_coupling_layer -> [(2, 64, 64)] [(ConditionNode 'fe7a40', 0)] [(256, 64, 64)]
Node 'e2bbc0': [(2, 64, 64)] -> Fixed1x1Conv -> [(2, 64, 64)] [] []
Node 'conv_3': [(2, 64, 64)] -> subnet_coupling_layer -> [(2, 64, 64)] [(ConditionNode 'fe7a40', 0)] [(256, 64, 64)]
Node 'e2bf50': [(2, 64, 64)] -> Fixed1x1Conv -> [(2, 64, 64)] [] []
Node 'reshape': [(2, 64, 64)] -> IRevNetDownsampling -> ((8, 32, 32),) [] []
Node 'e2bfe0': [(8, 32, 32)] -> Fixed1x1Conv -> [(8, 32, 32)] [] []
Node 'e7c0e0

In [9]:
combined_model

WrappedModel(
  (feature_network): KitModel(
    (bw_conv1_1): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv1_2): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (conv1_2norm): BatchNorm2d(64, eps=9.999999747378752e-06, momentum=0.1, affine=True, track_running_stats=True)
    (conv2_1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv2_2): Conv2d(128, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (conv2_2norm): BatchNorm2d(128, eps=9.999999747378752e-06, momentum=0.1, affine=True, track_running_stats=True)
    (conv3_1): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv3_2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv3_3): Conv2d(256, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (conv3_3norm): BatchNorm2d(256, eps=9.999999747378752e-06, momentum=0.1, affine=True, track_running_stats=True)
    (conv4_1): Conv2d(256, 512